In [ ]:
from scipy.spatial.distance import euclidean    
import pickle
import numpy as np
from skimage import novice
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from guppy import hpy
from PIL import ImageFile
from mpl_toolkits.mplot3d import Axes3D
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# Memory profiling
hp = hpy()

In [ ]:
# Constants
DISTANCE_SMOOTHING = 0.1
EMOTION_SCALING = 200.0
RADIUS_SCALING = 0.1
FROM_CENTER_SMOOTHING = 0.01

In [ ]:
# Training phase paths
TRAIN_BASE_PATH = 'Data/Train'
TRAIN_FACES_BASE_PATH = 'Data/Indi-Face-Predictions/faces/Train'

POSITIVE_TRAIN_BASE_PATH = os.path.join(TRAIN_BASE_PATH, 'Positive')
POSITIVE_TRAIN_FACES_BASE_PATH = os.path.join(TRAIN_FACES_BASE_PATH, 'positive', 'positive')
NEGATIVE_TRAIN_BASE_PATH = os.path.join(TRAIN_BASE_PATH, 'Negative')
NEGATIVE_TRAIN_FACES_BASE_PATH = os.path.join(TRAIN_FACES_BASE_PATH, 'negative', 'negative')
NEUTRAL_TRAIN_BASE_PATH = os.path.join(TRAIN_BASE_PATH, 'Neutral')
NEUTRAL_TRAIN_FACES_BASE_PATH = os.path.join(TRAIN_FACES_BASE_PATH, 'neutral', 'neutral')

# Training phase paths
TRAIN_DUMP_BASE_PATH = 'Data/Heatmaps/TrainGaussianNormalizedCenter'
TRAIN_DUMP_FACES_BASE_PATH = 'Data/Notebooks/Indi-Face-Predictions/faces/Heatmaps'

POSITIVE_DUMP_TRAIN_BASE_PATH = os.path.join(TRAIN_DUMP_BASE_PATH, 'Positive')
POSITIVE_DUMP_TRAIN_FACES_BASE_PATH = os.path.join(TRAIN_DUMP_FACES_BASE_PATH, 'positiveLinear')
NEGATIVE_DUMP_TRAIN_BASE_PATH = os.path.join(TRAIN_DUMP_BASE_PATH, 'Negative')
NEGATIVE_DUMP_TRAIN_FACES_BASE_PATH = os.path.join(TRAIN_DUMP_FACES_BASE_PATH, 'negativeLinear')
NEUTRAL_DUMP_TRAIN_BASE_PATH = os.path.join(TRAIN_DUMP_BASE_PATH, 'Neutral')
NEUTRAL_DUMP_TRAIN_FACES_BASE_PATH = os.path.join(TRAIN_DUMP_FACES_BASE_PATH, 'neutralLinear')

In [ ]:
# Test data paths
TEST_BASE_PATH = 'Data/TestImages_Latest'
TEST_FACES_BASE_PATH = 'Data/Test'

POSITIVE_TEST_BASE_PATH = os.path.join(TEST_BASE_PATH, 'Positive')
POSITIVE_TEST_FACES_BASE_PATH = os.path.join(TEST_FACES_BASE_PATH, 'Positive')
NEGATIVE_TEST_BASE_PATH = os.path.join(TEST_BASE_PATH, 'Negative')
NEGATIVE_TEST_FACES_BASE_PATH = os.path.join(TEST_FACES_BASE_PATH, 'Negative')
NEUTRAL_TEST_BASE_PATH = os.path.join(TEST_BASE_PATH, 'Neutral')
NEUTRAL_TEST_FACES_BASE_PATH = os.path.join(TEST_FACES_BASE_PATH, 'Neutral')

# Validate Dump paths 
TEST_DUMP_BASE_PATH = 'Data/Heatmaps/Test-Center-Normed'
TEST_DUMP_FACES_BASE_PATH = 'Data/Indi-Face-Predictions/faces/Heatmaps'

POSITIVE_DUMP_TEST_BASE_PATH = os.path.join(TEST_DUMP_BASE_PATH, 'Positive')
POSITIVE_DUMP_TEST_FACES_BASE_PATH = os.path.join(TEST_DUMP_FACES_BASE_PATH, 'positive')
NEGATIVE_DUMP_TEST_BASE_PATH = os.path.join(TEST_DUMP_BASE_PATH, 'Negative')
NEGATIVE_DUMP_TEST_FACES_BASE_PATH = os.path.join(TEST_DUMP_FACES_BASE_PATH, 'negative')
NEUTRAL_DUMP_TEST_BASE_PATH = os.path.join(TEST_DUMP_BASE_PATH, 'Neutral')
NEUTRAL_DUMP_TEST_FACES_BASE_PATH = os.path.join(TEST_DUMP_FACES_BASE_PATH, 'neutral')

In [ ]:
# Validation data paths
VALIDATE_BASE_PATH = 'Data/Validate'
VALIDATE_FACES_BASE_PATH = 'Data/Indi-Face-Predictions/faces/Validate'

POSITIVE_VALIDATE_BASE_PATH = os.path.join(VALIDATE_BASE_PATH, 'Positive')
POSITIVE_VALIDATE_FACES_BASE_PATH = os.path.join(VALIDATE_FACES_BASE_PATH, 'Positive')
NEGATIVE_VALIDATE_BASE_PATH = os.path.join(VALIDATE_BASE_PATH, 'Negative')
NEGATIVE_VALIDATE_FACES_BASE_PATH = os.path.join(VALIDATE_FACES_BASE_PATH, 'Negative')
NEUTRAL_VALIDATE_BASE_PATH = os.path.join(VALIDATE_BASE_PATH, 'Neutral')
NEUTRAL_VALIDATE_FACES_BASE_PATH = os.path.join(VALIDATE_FACES_BASE_PATH, 'Neutral')

# Validate Dump paths 
VALIDATE_DUMP_BASE_PATH = 'Data/Heatmaps/ValidateGaussianNormalizedCenter'
VALIDATE_DUMP_FACES_BASE_PATH = 'Data/Indi-Face-Predictions/faces/Heatmaps'

POSITIVE_DUMP_VALIDATE_BASE_PATH = os.path.join(VALIDATE_DUMP_BASE_PATH, 'Positive')
POSITIVE_DUMP_VALIDATE_FACES_BASE_PATH = os.path.join(VALIDATE_DUMP_FACES_BASE_PATH, 'positive')
NEGATIVE_DUMP_VALIDATE_BASE_PATH = os.path.join(VALIDATE_DUMP_BASE_PATH, 'Negative')
NEGATIVE_DUMP_VALIDATE_FACES_BASE_PATH = os.path.join(VALIDATE_DUMP_FACES_BASE_PATH, 'negative')
NEUTRAL_DUMP_VALIDATE_BASE_PATH = os.path.join(VALIDATE_DUMP_BASE_PATH, 'Neutral')
NEUTRAL_DUMP_VALIDATE_FACES_BASE_PATH = os.path.join(VALIDATE_DUMP_FACES_BASE_PATH, 'neutral')

In [ ]:
PAPER_FACES_PATH = 'Data/Paper/pics/faces/'
PAPER_PICS_PATH = 'Data/Paper/pics/pics/'

In [ ]:
# Get center and radius using rectangular points
def get_center_and_radius(top_left, bottom_right):
    x0, y0 = top_left
    x1, y1 = bottom_right
    # Mid-point of the diagonal gives the center
    center = ((x0 + x1) / 2.0, (y0 + y1) / 2.0)
    
    # Approximation of distance as 1 / 2 the distance  
    # between diagonal vertices of the rectangle (~square)
    diameter = euclidean(top_left, bottom_right)
    radius = diameter / 2.0
    
    return (center, radius)

In [ ]:
# Test center and radius function
print get_center_and_radius((0, 3), (3, 0))   #((1.5, 1.5), 2.12)
print get_center_and_radius((0, 3), (4, 0))   #((2.0, 1.5), 2.50)
print get_center_and_radius((2, 18), (14, 7)) #((8.0, 12.5), 8.14)

In [ ]:
def combine_emotion_vectors(vectors):
    # Average the five vectors and obtain scores for +ve, -ve and neutral emotions
    summation = np.zeros(7)
    for i in xrange(5):
        emo_vector = vectors[i][0]
        summation += emo_vector
    average = summation / 5.0
    
    positive = average[3]
    negative = np.sum(average[:3] + average[5]) / 4.0
    neutral = average[4]
    
    # Return in alphabetical order to make it easier to remember
    return (negative, neutral, positive)

In [ ]:
def process_face_heatmaps(face_heatmaps, path_to_save):
    heatmap_size = face_heatmaps[0].shape
    combination = np.zeros(heatmap_size)
    for hmap in face_heatmaps:
        combination += hmap
    plt.imsave(path_to_save + '.png', combination.astype(int))

In [ ]:
def make_linear(width, height, center=None, radius=3):
    
    x = np.arange(0, width, 1, float)
    y = np.arange(0, height, 1, float)
    y.shape = (height, 1)
    
    if center is None:
        x0 = width // 2
        y0 = height // 2
    else:
        x0, y0 = center
    distance = (x - x0) ** 2 + (y - y0) ** 2
    linear = (0.1 * abs(x - x0)) + (0.1 * abs(y - y0))
    inverse_linear = 1.0 / linear
    inverse_linear[np.isinf(inverse_linear)] = 1.0
    return inverse_linear

In [ ]:
def make_gaussian(width, height, center=None, fwhm = 3):
    """ Make a square gaussian kernel.
    fwhm is full-width-half-maximum, which
    can be thought of as an effective radius.
    """

    x = np.arange(0, width, 1, float)
    y = np.arange(0, height, 1, float)
    y.shape = (height, 1)
    
    if center is None:
        x0 = y0 = size // 2
    else:
        x0, y0 = center
    
    return np.exp(-4*np.log(2) * ((DISTANCE_SMOOTHING * (x-x0)) ** 2 + (DISTANCE_SMOOTHING * (y-y0)) **2) / fwhm**2)

In [ ]:
def create_3d_plot(numpy_array, destination, x_label, y_label, z_label):
    print numpy_array.shape
    fig = plt.figure()
    xs, ys = numpy_array.shape

    x = np.linspace(0, xs, xs)
    y = np.linspace(0, ys, ys)
    X, Y = np.meshgrid(y, x)
    
    print X.shape
    print Y.shape
    fig = plt.figure(figsize=(15, 12))
    ax = fig.gca(projection='3d')
    ax.plot_surface(X, Y, numpy_array, cmap='jet',linewidth=0)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_zlabel(z_label)
    
    plt.savefig(destination)
    plt.clf()

In [ ]:
global_image = None
def process_pickles(id_coord_pkl, emotion_pkl, image_base_path, subimage_base_path, 
                    image_destination_base_path, subimage_destination_base_path):
    global global_image
    # See if the pickles are of same length
    assert len(id_coord_pkl[0]) == len(emotion_pkl), 'Pickle length mismatch'
    image_names = id_coord_pkl[0]
    subimage_names = id_coord_pkl[1]
    coordinates = id_coord_pkl[2]
    
    current_image = None
    current_img_object = None
    current_img_size = None
    current_img_center = None
    new_image_flag = None
    image_heatmaps = list()
    for i in tqdm(xrange(len(image_names))):
        # See if it's a new sub-image or the old one
        image_name = image_names[i]
        
        if image_name != current_image:
            if current_image is not None:                
                path_to_save = os.path.join(image_destination_base_path, current_image)
                plt.imsave(path_to_save + '.png', combination.astype(int))
            current_image = image_name
            current_img_object = novice.open(os.path.join(image_base_path, current_image))
            current_img_size = current_img_object.size
            current_img_center = current_img_size[0] // 2, current_img_size[1] // 2
            new_image_flag = True
        else:
            new_image_flag = False
            
        subimage_name = subimage_names[i]
        # subimage_object = novice.open(os.path.join(subimage_base_path, subimage_name))
        # subimage_object_size = subimage_object.size
            
        subimage_coordinates = coordinates[i]
        y1 = subimage_coordinates[0]
        y0 = subimage_coordinates[1]
        x0 = subimage_coordinates[2]
        x1 = subimage_coordinates[3]
        
#         subimage_size_calculated = (abs(x1 - x0), abs(y1 - y0))
#         Sanity Check: The size of the subimage must be the same as in the pickle
#         assert subimage_object_size == subimage_size_calculated, '''Image on disk: {0}. 
#         Image in pickle: {1}.
#         Index: {2}. 
#         Pickle Image: {3}
#         Disk Image: {4}
#         Main Image: {5}
#         Coords: {6}'''.format(subimage_object_size, subimage_size_calculated, i, subimage_name, 
#                              os.path.join(subimage_base_path, subimage_name), current_image,
#                              subimage_coordinates)
        
        center, radius = get_center_and_radius((x0, y1), (x1, y0))
        distance_from_img_center = euclidean(center, current_img_center)
#         print 'Center:', center, 'Image Center:', current_img_center
#         print 'Distance:', distance_from_img_center
        distance_from_center_smoothed = FROM_CENTER_SMOOTHING * distance_from_img_center
        distance_from_center_inverse = 1.0 / distance_from_center_smoothed
        scaled_radius = RADIUS_SCALING * radius
        emotion_vectors = emotion_pkl[i]
        (negative, positive, neutral) = combine_emotion_vectors(emotion_vectors)
        
        # Now we have everything we need to construct a heatmap
        # print 'positive: ', positive, 'negative: ', negative, 'neutral: ', neutral, 'center:', center, 'radius: ', radius 
#         pmap = distance_from_center_inverse * EMOTION_SCALING * positive * make_gaussian(current_img_size[0], current_img_size[1], center, scaled_radius)
#         omap = distance_from_center_inverse * EMOTION_SCALING * neutral * make_gaussian(current_img_size[0], current_img_size[1], center, scaled_radius)
#         nmap = distance_from_center_inverse * EMOTION_SCALING * negative * make_gaussian(current_img_size[0], current_img_size[1], center, scaled_radius)
        
        pmap = EMOTION_SCALING * positive * make_gaussian(current_img_size[0], current_img_size[1], center, scaled_radius)
        omap = EMOTION_SCALING * neutral * make_gaussian(current_img_size[0], current_img_size[1], center, scaled_radius)
        nmap = EMOTION_SCALING * negative * make_gaussian(current_img_size[0], current_img_size[1], center, scaled_radius)
        
        
        # Contruct an image using the heatmap
        # R: Negative, G: Neutral, B: Positive
        image = np.dstack((nmap, omap, pmap))
        plt.imsave(os.path.join(subimage_destination_base_path, subimage_name + '_int_pmap.png'), pmap.astype(int))
        plt.imsave(os.path.join(subimage_destination_base_path, subimage_name + '_int_omap.png'), omap.astype(int))
        plt.imsave(os.path.join(subimage_destination_base_path, subimage_name + '_int_nmap.png'), nmap.astype(int))
        plt.imsave(os.path.join(subimage_destination_base_path, subimage_name + '_combined.png'), image.astype(int))
        
        # create_3d_plot(pmap, os.path.join(subimage_destination_base_path, subimage_name + '_gaussian_pos.png'), 'Width', 'Height', 'Positive Emotion')
        # create_3d_plot(nmap, os.path.join(subimage_destination_base_path, subimage_name + '_gaussian_neg.png'), 'Width', 'Height', 'Negative Emotion')
        # create_3d_plot(omap, os.path.join(subimage_destination_base_path, subimage_name + '_gaussian_neu.png'), 'Width', 'Height', 'Neutral Emotion')
        if new_image_flag:
            combination = image
            #print combination
        else:
            combination += image
            #print combination
#         print type(image)
#         print image.shape
        # Write the image on to the disk and add it to the list for further processing
        # heatmaps.append(image)
        
        # https://stackoverflow.com/questions/31544130/saving-an-imshow-like-image-while-preserving-resolution
        # plt.imsave(os.path.join(subimage_destination_base_path, subimage_name + '.png'), image.astype(int))
    
    # Save the pickle object in the end



## Use saved predictions
Each of the pickles used below were obtained by extracting faces from the images and predicting emotions using models by [Levi and Hassner](https://www.openu.ac.il/home/hassner/projects/cnn_emotions/)

In [ ]:
positive_coord_pickle = pickle.load(open('positive_details_.obj', 'rb'))
positive_emo_pickle = pickle.load(open('train_positive_predictions.pkl', 'rb'))

process_pickles(positive_coord_pickle, positive_emo_pickle, POSITIVE_TRAIN_BASE_PATH, 
                POSITIVE_TRAIN_FACES_BASE_PATH, POSITIVE_DUMP_TRAIN_BASE_PATH, 
                POSITIVE_DUMP_TRAIN_FACES_BASE_PATH)

In [ ]:
neutral_coord_pickle = pickle.load(open('neutral_details.obj', 'rb'))
neutral_emo_pickle = pickle.load(open('train_neutral_predictions.pkl', 'rb'))

process_pickles(neutral_coord_pickle, neutral_emo_pickle,  NEUTRAL_TRAIN_BASE_PATH, 
                NEUTRAL_TRAIN_FACES_BASE_PATH, NEUTRAL_DUMP_TRAIN_BASE_PATH, 
                NEUTRAL_DUMP_TRAIN_FACES_BASE_PATH)

In [ ]:
negative_coord_pickle = pickle.load(open('negative_details_.obj', 'rb'))
negative_emo_pickle = pickle.load(open('train_negative_predictions.pkl', 'rb'))

process_pickles(negative_coord_pickle, negative_emo_pickle,  NEGATIVE_TRAIN_BASE_PATH, 
                NEGATIVE_TRAIN_FACES_BASE_PATH, NEGATIVE_DUMP_TRAIN_BASE_PATH, 
                NEGATIVE_DUMP_TRAIN_FACES_BASE_PATH)

In [ ]:
positive_coord_pickle = pickle.load(open('positive_details_validation_data.obj', 'rb'))
positive_emo_pickle = pickle.load(open('validate_positive_predictions.pkl', 'rb'))

process_pickles(positive_coord_pickle, positive_emo_pickle, POSITIVE_VALIDATE_BASE_PATH, 
                POSITIVE_VALIDATE_FACES_BASE_PATH, POSITIVE_DUMP_VALIDATE_BASE_PATH, 
                POSITIVE_DUMP_VALIDATE_FACES_BASE_PATH)

In [ ]:
negative_coord_pickle = pickle.load(open('negative_details_validation_data.obj', 'rb'))
negative_emo_pickle = pickle.load(open('validate_negative_predictions.pkl', 'rb'))

process_pickles(negative_coord_pickle, negative_emo_pickle, NEGATIVE_VALIDATE_BASE_PATH, 
                NEGATIVE_VALIDATE_FACES_BASE_PATH, NEGATIVE_DUMP_VALIDATE_BASE_PATH, 
                NEGATIVE_DUMP_VALIDATE_FACES_BASE_PATH)

In [ ]:
# To read memory footprint
# hp.heap()

In [ ]:
neutral_coord_pickle = pickle.load(open('neutral_details_validation_data.obj', 'rb'))
neutral_emo_pickle = pickle.load(open('validate_neutral_predictions.pkl', 'rb'))

process_pickles(neutral_coord_pickle, neutral_emo_pickle, NEUTRAL_VALIDATE_BASE_PATH, 
                NEUTRAL_VALIDATE_FACES_BASE_PATH, NEUTRAL_DUMP_VALIDATE_BASE_PATH, 
                NEUTRAL_DUMP_VALIDATE_FACES_BASE_PATH)

In [ ]:
# Test data
coord_pickle = pickle.load(open('details_test_data_new.obj', 'rb'))
emo_pickle = pickle.load(open('test_predictions.pkl', 'rb'))

process_pickles(coord_pickle, emo_pickle, NEGATIVE_TEST_BASE_PATH, 
                NEGATIVE_TEST_FACES_BASE_PATH, NEGATIVE_DUMP_TEST_BASE_PATH, 
                NEGATIVE_DUMP_TEST_FACES_BASE_PATH)

In [ ]:
# Paper data
coord_pickle = pickle.load(open('paper_data.obj', 'rb'))
emo_pickle = pickle.load(open('paper_predictions.pkl', 'rb'))

process_pickles(coord_pickle, emo_pickle, 'Data/pics/pics', 
                'Data/pics/faces', 'Data/pics/combined_heatmap', 
                'Data/pics/indi_heat_maps')